In [ ]:
import numpy as np
import pandas as pd
import hdbscan
from hdbscan.validity import validity_index
from sklearn.manifold import trustworthiness
from sklearn.metrics import accuracy_score


In [ ]:
df = pd.read_parquet('./data/Gold/Gold_embedding_imagen_marca.parquet')
df

In [ ]:
embeddings = df[['UMAP_1', 'UMAP_2', 'UMAP_3']].values
labels = df['clusters_hdbscan'].values

embeddings_originales = np.load('./data/Silver/embeddings.npy')

In [ ]:
##### evalua calidad embedding UMAP


### se necesita hacerun submuuestreo porque crea una matriz tan grande que necesita 129 gigas. 

# aprox 25% de las muestras
n_sample = 32870  
N = embeddings.shape[0]  
np.random.seed(69) 
sample_idx = np.random.choice(N, size=n_sample, replace=False)


# Aplicar los índices a ambos arrays
embedded_muestro = embeddings[sample_idx]
original_muestreo = embeddings_originales[sample_idx]


trust_score = trustworthiness(original_muestreo ,
                              embedded_muestro,
                              n_neighbors=100 ### mismo que en UMAP
                             )
print(f"Trustworthiness: {trust_score:.2f}")

In [ ]:
##quiere labels numericos 
categorias = set(labels)

# Crear mapping
categorias_lista = sorted(categorias)
categoria2id = {}
i = 0
for cat in categorias_lista:
    if cat == "Desconectado":
        categoria2id[cat] = -1
    else:
        categoria2id[cat] = i
        i += 1
labels_numericos = [categoria2id[label] for label in labels]

In [ ]:
#### evaluación clustering HDBSCAN 
# se emplea Density-Based Clustering Validation comoo métrica de evaluación, para clusters de densidad. 
#https://github.com/christopherjenness/DBCV/blob/master/DBCV/DBCV.py  
# -->> esta implementación es miuy lenta y para este dataset es tan grande que se opta por al de hdbscan que se basa en cython, aunque es menos precisa.
# 
#se hace sobre el embedding final



score = validity_index(
                       np.array(embeddings, dtype=np.float64), 
                       np.array(labels_numericos)
                       )
print("DBCV:", score)

In [ ]:
df_sin_ou t= df[df['clusters_hdbscan']!='Desconectado']
df_sin_out

In [ ]:
embeddings_wo = df_sin_out[['UMAP_1', 'UMAP_2', 'UMAP_3']].values
labels_wo = df_sin_out['clusters_hdbscan'].values

##quiere labels numericos 
categorias = set(labels)

# Crear mapping
categorias_lista = sorted(categorias)
categoria2id = {}
i = 0
for cat in categorias_lista:
    if cat == "Desconectado":
        categoria2id[cat] = -1
    else:
        categoria2id[cat] = i
        i += 1
labels_numericos_wo = [categoria2id[label] for label in labels_wo ]

score = validity_index(
                       np.array(embeddings_wo, dtype=np.float64), 
                       np.array(labels_numericos_wo )
                       )
print("DBCV sin desconectados:", score)

In [ ]:
###############  ACONTINUACIÓN SE VA A ANALIZAR EL ACCURACY DE LA CLASIFICACIÓN SOBRE LA TAXONOMIA DE MERCADONA

df = pd.read_parquet('./data/Silver/Cleaned_data_features_productos_categorias.parquet')
df

In [ ]:
#subset
subset = df[df['sentimiento_producto']=='SI'].sample(n=100, random_state=69)
subset.to_excel('manual_label_categorias.xlsx', index=False)

#se hace un manual label 
#valores binarios:
#    1 bien categorizado 
#    0 No está bien clasificado.
## si habla de productos de navidad se deja a 1 porque no se puede categorizar con el catalogo actual y no se considera que falla
## Notas: 
## hummus falla bastante
## cuando se habla algo  de hacendado (despectivo) falla.

In [ ]:
df_manual = pd.read_excel('./data/Silver/manual_label_categorias_manual_label.xlsx')
df_manual['automatic_label'] = [1] * len(df_manual)
df_manual = df_manual[['manual_label','automatic_label']]
df_manual = df_manual.fillna(1)
#df_manual = df_manual.dropna(subset=['manual_label'])
df_manual['manual_label'] = df_manual['manual_label'].astype(int)
df_manual

In [ ]:
y_true = df_manual['manual_label']
y_pred = df_manual['automatic_label']

# Accuracy
acc = accuracy_score(y_true, y_pred)
print("Accuracy:", acc)